In [1]:
from model_builder.GIS import tinyGIS
TG = tinyGIS()
import pandas as pd
import numpy as np
from typing import Dict, Union
from itertools import product
import os
import xarray as xr
import re

tinyGIS initiated


## Preparation of modfied soil files with dimension n, subbasin, and dimension s as soil

In [2]:
# read the soil file, get it majority and save
file_path = '/home/shg096/scratch/test/gistool_outputs/soil_classes/'
file_name_csv = 'domain_stats_soil_classes.csv'
file_name_nc = 'domain_stats_soil_classes.nc'
num_soil = 12
soil_name = ['1 clay', '2 silty clay', '3 sandy clay', '4 clay loam','5 silty clay loam',\
             '6 sandy clay loam', '7 loam', '8 silty loam', '9 sandy loam', '10 silt',\
             '11 loamy sand', '12 sand']

In [3]:
# read the file
soil_type = pd.read_csv(file_path+file_name_csv)

# drop the 0 soil type which is mostly to represent water
soil_type = soil_type.drop(columns = 'frac_0')

# add non existing columns and one line of non zero values
missing_columns = [f"frac_{i}" for i in range(1, num_soil+1) if f"frac_{i}" not in soil_type.columns]
for col in missing_columns:
    soil_type[col] = 0
    
# Sort columns that start with "frac_" based on the numbers at the end
frac_columns = [col for col in soil_type.columns if re.match(r'^frac_\d+$', col)]
frac_columns.sort(key=lambda x: int(re.search(r'\d+$', x).group()))

# Reorder DataFrame columns with sorted "frac_" columns
sorted_columns = [col for col in soil_type.columns if col not in frac_columns] + frac_columns
soil_type = soil_type.reindex(columns=sorted_columns)

# set the fraction to majority 1 and other to zero
soil_type   =  TG.manipulating_fractions(soil_type,
                                         df_mapping={'prefix': 'frac_'},
                                         action = 'majority')

# report the soil type with majority in majority column
soil_type   = TG.report_max_frac(soil_type,
                                 df_mapping={'prefix': 'frac_'})

# save the modified file into the csv
soil_type.to_csv(file_path+'modified_'+file_name_csv)

## xarray object and netcdf
# set index to of COMID and name it n
soil_type = soil_type.set_index('COMID')
soil_type.index.name = 'n'

# Convert majority to xarray DataArray
data_array_majority = xr.DataArray(soil_type['majority'].values, dims=('n',), coords={'n': soil_type.index})

# create xarray for majority and frac_
data_array_frac = xr.DataArray(soil_type.filter(like='frac').values, dims=('n', 's'), coords={'n': soil_type.index, 's': soil_name})

# Combine into xarray Dataset
ds = xr.Dataset({'soil_majority': data_array_majority, 'soil_frac': data_array_frac})

# save the file
if os.path.isfile(file_path+'modified_'+file_name_nc):
    os.remove(file_path+'modified_'+file_name_nc)
ds.to_netcdf(file_path+'modified_'+file_name_nc)


frac_3


## Preparation of modfied land cover files with dimension n, subbasin, and dimension l as land cover

In [4]:
# read the soil file, get it majority and save
file_path = '/home/shg096/scratch/test/gistool_outputs/landsat/'
file_name_csv = 'domain_stats_NA_NALCMS_landcover_2020_30m.csv'
file_name_nc = 'domain_stats_NA_NALCMS_landcover_2020_30m.nc'
num_land_cover = 19
landcover_type = pd.read_csv(file_path+file_name_csv)
LULC_name =  ['1 Temperate or sub-polar needleleaf forest',\
              '2 Sub-polar taiga needleleaf forest',\
              '3 Tropical or sub-tropical broadleaf evergreen forest',\
              '4 Tropical or sub-tropical broadleaf deciduous forest',\
              '5 Temperate or sub-polar broadleaf deciduous forest',\
              '6 Mixed forest',\
              '7 Tropical or sub-tropical shrubland',\
              '8 Temperate or sub-polar shrubland',\
              '9 Tropical or sub-tropical grassland',\
              '10 Temperate or sub-polar grassland',\
              '11 Sub-polar or polar shrubland-lichen-moss',\
              '12 Sub-polar or polar grassland-lichen-moss',\
              '13 Sub-polar or polar barren-lichen-moss',\
              '14 Wetland',\
              '15 Cropland',\
              '16 Barren lands',\
              '17 Urban',\
              '18 Water',\
              '19 Snow and Ice']

In [5]:
# add non existing columns and one line of non zero values
missing_columns = [f"frac_{i}" for i in range(1, num_land_cover+1) if f"frac_{i}" not in landcover_type.columns]
for col in missing_columns:
    landcover_type[col] = 0
    
# Sort columns that start with "frac_" based on the numbers at the end
frac_columns = [col for col in landcover_type.columns if re.match(r'^frac_\d+$', col)]
frac_columns.sort(key=lambda x: int(re.search(r'\d+$', x).group()))

# Reorder DataFrame columns with sorted "frac_" columns
sorted_columns = [col for col in landcover_type.columns if col not in frac_columns] + frac_columns
landcover_type = landcover_type.reindex(columns=sorted_columns)

# smooth land cover and remove below 5% land cover and renormalize
landcover_type =  TG.manipulating_fractions(landcover_type,
                                            df_mapping={'prefix': 'frac_'},
                                            action = 'normalize',
                                            minimum_value = 0.05)

# get the majority of the soil type for each subbasin
landcover_type   =  TG.report_max_frac(landcover_type,
                                        df_mapping={'prefix': 'frac_'})

# # replace the first line zeros with minimum values for CLASS to run
# for col in frac_columns:
#     if landcover_type.loc[0, col] < 0.00001:
#         landcover_type.loc[0, col] = 0.00001
        
# save the modified file into the csv
landcover_type.to_csv(file_path+'modified_'+file_name_csv)

# set index to of COMID and name it n
landcover_type = landcover_type.set_index('COMID')
landcover_type.index.name = 'n'

# Convert majority to xarray DataArray
data_array_majority = xr.DataArray(landcover_type['majority'].values, dims=('n',), coords={'n': landcover_type.index})

# create xarray for majority and frac_
data_array_frac = xr.DataArray(landcover_type.filter(like='frac').values, dims=('n', 'l'), coords={'n': landcover_type.index, 'l': LULC_name})

# Combine into xarray Dataset
ds = xr.Dataset({'LULC_majority': data_array_majority, 'LULC_frac': data_array_frac})


if os.path.isfile(file_path+'modified_'+file_name_nc):
    os.remove(file_path+'modified_'+file_name_nc)
ds.to_netcdf(file_path+'modified_'+file_name_nc)

ds

frac_1


<xarray.Dataset>
Dimensions:        (n: 410, l: 19)
Coordinates:
  * n              (n) int64 71022153 71022160 71022164 ... 71039250 71039266
  * l              (l) <U53 '1 Temperate or sub-polar needleleaf forest' ... ...
Data variables:
    LULC_majority  (n) object '15' '15' '15' '15' '15' ... '16' '16' '1' '16'
    LULC_frac      (n, l) float64 0.0 0.0 0.0 0.0 0.0 ... 0.4622 0.0 0.1151 0.0

## mean elevation based on dimension n

In [6]:
## add other information to ds
file_path = '/home/shg096/scratch/test/gistool_outputs/merit_hydro/'
file_name_csv = 'domain_stats_elv.csv'
file_name_nc = 'domain_stats_elv.nc'

elev = pd.read_csv(file_path+file_name_csv)
elev.index = elev['COMID'].astype(int)
elev['mean_elev'] = elev['mean']
elev = elev.drop(columns=['COMID','min','max','mean','median'])


elev.index.name = 'n'
ds_elev = elev.to_xarray()
ds_elev['mean_elev'].attrs['unit'] = 'm'

if os.path.isfile(file_path+file_name_nc):
    os.remove(file_path+file_name_nc)
ds_elev.to_netcdf(file_path+file_name_nc)

ds_elev

<xarray.Dataset>
Dimensions:     (n: 410)
Coordinates:
  * n           (n) int64 71022153 71022160 71022164 ... 71039250 71039266
Data variables:
    Unnamed: 0  (n) int64 0 1 2 3 4 5 6 7 8 ... 402 403 404 405 406 407 408 409
    mean_elev   (n) int64 5 5 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5 5 5

## preparation of modfied soil files with dimension n, subbasin, and m as combination of soil and land cover

In [7]:
# call the function to create the merger of soil and land cover
path = '/home/shg096/scratch/test/gistool_outputs/combination/'
file_name_nc = 'modified_combination.nc'
soil_type = pd.read_csv('/home/shg096/scratch/test/gistool_outputs/soil_classes/modified_domain_stats_soil_classes.csv')
land_cover = pd.read_csv('/home/shg096/scratch/test/gistool_outputs/landsat/modified_domain_stats_NA_NALCMS_landcover_2020_30m.csv')

result,report, ds =  TG.intersect_df(soil_type, land_cover,
                                     df_mappings={'df1': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'soil'}, 
                                                  'df2': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'LULC'}},
                                     remove_zero_combinations = True)

# get the majority of cobination for each subbasin
result   =  TG.report_max_frac(result,
                               df_mapping={'prefix': 'comb_'})

# rename the index for the result
result.index.name = 'n'

# create xarray for frac_
data_array_frac = xr.DataArray(result.filter(like='comb').values, dims=('n', 'm'), coords={'n': result.index, 'm': result.filter(like='comb').columns})

# create xarray for majority
data_array_majority = xr.DataArray(result['majority'].values, dims=('n'), coords={'n': result.index})

# Combine into xarray Dataset
ds = xr.Dataset({'comb_frac': data_array_frac, 'comb_majority': data_array_majority})


# convert the value using the m
report = report.set_index('Combinations')
report.index.name = 'm'
ds_report = report.to_xarray()

ds['comb_soil'] = xr.DataArray(ds_report['soil'].values, dims=('m'))
ds['comb_LULC'] = xr.DataArray(ds_report['LULC'].values, dims=('m'))


if not os.path.isdir(path):
    os.makedirs(path)

if os.path.isfile(path+file_name_nc):
    os.remove(path+file_name_nc)
ds.to_netcdf(path+file_name_nc)

ds

The indexes of all DataFrames are exactly the same with the same order.
total number of non zero combinations:  15


<xarray.Dataset>
Dimensions:        (n: 410, m: 15)
Coordinates:
  * n              (n) int64 71022153 71022160 71022164 ... 71039250 71039266
  * m              (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
Data variables:
    comb_frac      (n, m) float64 0.0 0.0 0.0 0.07281 0.0 ... 0.0 0.0 0.0 0.0
    comb_majority  (n) object '0007' '0007' '0007' ... '0008' '0001' '0008'
    comb_soil      (m) object '3' '3' '3' '3' '3' '3' ... '3' '8' '8' '8' '8'
    comb_LULC      (m) object '1' '5' '6' '8' '10' ... '19' '1' '10' '16' '19'